# Close Probability Analysis Log
## Parse Qualtrics Output

#### Step 1: Import libraries, load file

In [1]:
import pandas as pd
import re

df=pd.read_csv('rawDatFewMore.csv',skiprows=[1,])
df=df.rename(columns={df.columns[0]:'subj_id'})

####Step 2: Split lists
Assuming you have an embeded data variable created by your Qualtrics survey, here `listNb` you can sort your different participants into blocks. This will extract each participant as a row, and since the columns of the other block will be empty you will want to drop those columns so as not to have empty/useless data.

You then store it in a dictionary `dic` which you will increment for each list.

Because pandas doesn't like to have column labels with the same name, and rightly so, it will add `.X` where `X` is a number to each duplicate column. This is going to be a problem later on when you're trying to match the column index with the sentence/row index from your *loop and merge key*.
We therefore incorporate a step where `re.sub()` will delete a specified pattern (regular expression) consisting of everything after and including the `.`. 

In [2]:
dic={}
for curList in xrange(1,5):
    dic[curList]=df.loc[df['listNb']==curList].dropna(1,how='all')
    cols=list(dic[curList].columns.values) #store columns in a list
    new_cols=[re.sub('\..*$', "", elem) for elem in cols] #make the resub go through each element of the list and do its thing
    dic[curList].columns=new_cols #replace columns back in df.


####Step 3: Perform Parsing
Here the steps are written in a function that melts and retrieves the information that we care about.
This function is then applied in the next code cell to the different dataframes in the dictionary for each list and appends the lists back together.

In [3]:
def melt_df_feed_info_from_loop_merge(df):
    
    #MELT | Transpose df into long format
    col_names=pd.Series(df.columns)
    valid_cols_mask=col_names.str.contains(colsToPull,regex=True)
    valid_cols = df.columns[valid_cols_mask]
    df=df[valid_cols]
    df=pd.melt(df,id_vars='subj_id',var_name='qual_col',value_name=measureName)

    #Extract useful information from qual_cols, and merge key info
    qualtrics_parsed=df.qual_col.str.extract('^(?P<measure>\w+)\((?P<row>\d+)\)$')
    qualtrics_parsed = qualtrics_parsed.convert_objects(convert_numeric=True)
    df = pd.concat([df, qualtrics_parsed], axis = 1)
    df = df.merge(loopmergeInfo, how='left')

    return df

In [4]:
df.head(1)

,subj_id,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,AdditCom(13),AdditCom(14),AdditCom(15),AdditCom(16),AdditCom(17),AdditCom(18),AdditCom(19),LocationLatitude,LocationLongitude,LocationAccuracy
0,R_vrAH2qrxu41QSv7,Default Response Set,Anonymous,NaN,NaN,84.241.214.178,0,01/02/16 20:07,01/02/16 20:16,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.366592,4.902695,-1


In [11]:
colsToPull='^subj_id|Diff'
measureName='Diff'

for curList in xrange(1,5):
    loopmergeInfo=pd.read_csv('loopmergeFewMore.csv')
    if len(dic[curList])>=1:
        if curList==1:
            reconst=melt_df_feed_info_from_loop_merge(dic[curList])
        else:
            reconst=reconst.append(melt_df_feed_info_from_loop_merge(dic[curList]))
    else:
        print('There was no data in list %d') % curList

There was no data in list 2
There was no data in list 3
There was no data in list 4


/Users/boutonnetbpa/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:12: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [12]:
reconst.head()

,subj_id,qual_col,Diff,measure,row,sentID
0,R_vrAH2qrxu41QSv7,Diff(1),6,Diff,1,143
1,R_6Pg4ql6MQumxZo9,Diff(1),6,Diff,1,143
2,R_1QaBlAsk46HI9NJ,Diff(1),6,Diff,1,143
3,R_pKtuHvEeh5mujgl,Diff(1),6,Diff,1,143
4,R_RrYmeJDnKB3hv7r,Diff(1),6,Diff,1,143


####Step 4: Save

In [13]:
reconst.to_csv('reconstFewMoreDiff.csv',index=0)

####Step 4: Analyse/Visualise Data

So far the analysis is written in R code, it basically consists of sumarising the file by sentence, completion and get a measure of popularity for each participant. These functions from the `ddply` package are likely to be operationalisable in `pandas` but I haven't worked it out yet.

Follow up analysis on R (using R script). Maybe this could be analysed using Python too, but at least it works so far.

## Merging DFs when several variables have been extracted

In [28]:
comp=pd.read_csv('fewMoreTestIsWord.csv')
comp=comp.drop(['measure','row','qual_col'],1)
nat=pd.read_csv('reconstFewMoreNatNess.csv')
nat=nat.drop(['measure','row','qual_col'],1)
diff=pd.read_csv('reconstFewMoreDiff.csv')
diff=diff.drop(['measure','row','qual_col'],1)
RT=pd.read_csv('reconstFewMoreRT.csv')
RT=RT.drop(['measure','row','qual_col'],1)

In [29]:
compNat=pd.merge(comp,nat,on=['subj_id','sentID'],how='left')
compNatDiff=pd.merge(compNat,diff,on=['subj_id','sentID'],how='left')
compNatDiffRT=pd.merge(compNatDiff,RT,on=['subj_id','sentID'],how='left')

In [30]:
compNatDiffRT.head(30)

,subj_id,Completion,sentID,compNoTrail,compLower,determiner,noun,isDet,isWord,NatNess,Diff,RT
0,R_vrAH2qrxu41QSv7,de trein,143,de trein,de trein,de,trein,True,True,5,6,11.771
1,R_6Pg4ql6MQumxZo9,de trein,143,de trein,de trein,de,trein,True,True,5,6,2.613
2,R_1QaBlAsk46HI9NJ,de fiets,143,de fiets,de fiets,de,fiets,True,True,5,6,6.412
3,R_pKtuHvEeh5mujgl,de auto,143,de auto,de auto,de,auto,True,True,5,6,5.574
4,R_RrYmeJDnKB3hv7r,de trein,143,de trein,de trein,de,trein,True,True,3,6,10.961
5,R_2alPe2ZkILnzWYp,de trein,143,de trein,de trein,de,trein,True,True,4,6,39.432
6,R_24zMzQY1U9LX5Gf,de trein,143,de trein,de trein,de,trein,True,True,3,6,6.684
7,R_2zAp7MZjWbJtouD,de trein,143,de trein,de trein,de,trein,True,True,1,6,12.664
8,R_1lgea30X5yRbWgb,Auto,143,Auto,auto,NaN,-99,False,False,2,6,10.986
9,R_1dg0dAPlymYfFcu,de trein,143,de trein,de trein,de,trein,True,True,5,6,1.819


In [31]:
compNatDiffRT.to_csv('./output/allVarsFewMore.csv',index=0)

## Pre-Analysis Cleanups

### Comment Analysis
To this file I then add a column (this was done in excel) that adds a 1 if the `additCom` column contains a comment and a 0 if it doesn't so that it can be sorted for quick review.

### Typo checks
Here's a list of steps to do:
- split completion into determiner and word.
- match whether answer is a word from corpus.
- correct for typos.
- Run actual cloze probability analyses.
- Obtain, from a corpus, the frequencies, letter number etc. of the most popular item and come up with words from that corpus that match these frequencies. These words will then have to be used in "silly" filler sentences.

In [39]:
import pandas as pd
import numpy as np

#dat=pd.read_csv('allVarsCommentReviewed.csv') #This may change to the more "final" file.
dat=pd.read_csv('reconstFewMore.csv')

compSplit=dat.Completion.str.extract('^(?P<determiner>\w+) (?P<noun>\w+)$')
dat=pd.concat([dat,compSplit],axis=1)
dat.head()

,subj_id,qual_col,Completion,measure,row,sentID,determiner,noun
0,R_1AeLfMZXZ76HTHz,Comp(1),sd,Comp,1,143,NaN,NaN
1,R_vrAH2qrxu41QSv7,Comp(1),de trein,Comp,1,143,de,trein
2,R_6Pg4ql6MQumxZo9,Comp(1),de trein,Comp,1,143,de,trein
3,R_1QaBlAsk46HI9NJ,Comp(1),de fiets,Comp,1,143,de,fiets
4,R_pKtuHvEeh5mujgl,Comp(1),de auto,Comp,1,143,de,auto


In [40]:
grouped=dat.groupby(['sentID','Completion'])
test=grouped['subj_id'].agg({'pop':np.size})#.reset_index()#.sort('pop')
g=test['pop'].groupby(level=0,group_keys=False)
res=g.apply(lambda x: x.order(ascending=False))
res=res.reset_index()

/Users/boutonnetbpa/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: order is deprecated, use sort_values(...)


In [41]:
res.head()

,sentID,Completion,0
0,143,de trein,27
1,143,De trein,4
2,143,de auto,3
3,143,trein,1
4,143,sd,1


### Detect Typos and Non-Words
#### Using Simple corpus matching
NOTE: The SUBTLEX-NL corpus, has been edited so that it would only contain words, and spaces were removed. This is because sometimes we were getting false matches.

In [42]:
import pandas as pd
import numpy as np

dat=pd.read_csv('reconstFewMore.csv') #This may change to the more "final" file.

compRmTrail=dat.Completion.str.rstrip()
compRmTrail.name='compNoTrail'
dat=pd.concat([dat,compRmTrail],axis=1)
compSplit=dat.compNoTrail.str.extract('^(?P<determiner>\w+) (?P<noun>\w+)$')
dat=pd.concat([dat,compSplit],axis=1)

corpus=pd.read_table('SUBTLEX-NL.txt')
corpus.head()
detCorpus=['de','het','De','Het']

dat['noun']=dat.noun.fillna(-99)
dat['isDet']=dat.determiner.isnull()
dat['isDet']=dat.determiner.isin(detCorpus)
dat['isWord']=dat.noun.isin(corpus.Word)
dat.to_csv('fewMoreTestIsWord.csv',index=0)

In [43]:
#test=dat.Completion.str.rstrip()
#test.iloc[24]
compRmTrail.name='compNoTrail'
compRmTrail.head()

0          sd
1    de trein
2    de trein
3    de fiets
4     de auto
Name: compNoTrail, dtype: object

- Get participants to correct typos.
- Run actual close probability (popularity) analysis
- Analyse further
- Extract top value for each sentence and merge with sentence file if necessary later.

### Clearing aberrant answers
- 3/11/15: done manually, by deleting non-words after typo correction
- Create two cleaned up versions:
    - One with no change in content destined to the fMRI design.    
    **SUCCESS RATE**: **107** sentences that are above or equal to 70% cloze probability.
    
    - One with gender preseving alternate candidates (*laissez-faire* version)
    *maybe this is not needed since it might be easier for Mareike to just generate new sentences.

## Todo
### Further stimuli creation
- Since the success rate of all sentences **107** we need at least **13** more sentences created. **>>Mareike is on it**
    - these new sentences should be submitted to a cloze probability test.
- We now need a bunch of filler sentences.

### On current datafiles
- The results from the cloze test needs to:
    - be merged with the **full sentence**. **>>done**
    - be merged with the **frequency of the best completion** from a corpus. **>>done**
- Make sure the candidate sentences that are >70% are "het" or "de" answers. **>>done**
- Separate determiner from noun in the completion answer so that it's easier for sorting after. **>>done**
